## Demografische Entwicklung Wiens seit 2008: Analyse der
## Bevölkerungsstruktur und Geburtenentwicklung auf
## Bezirksebene
<u>**Big Data Projekt von:**</u>
<br>
Johannes Reitterer <br>
Johannes Mantler <br>
Nicolas Nemeth <br>
<br>

**Imports**

In [ ]:
import pandas as pd
import io
import requests

### Data Preprocessing (Extraction, Transform, Load) <br>


### 1. Verwendete Datenquellen
Wir nutzen zwei offene Datensätze (Open Government Data) der Stadt Wien:

* **Datensatz A: Bevölkerung nach Geburtsbundesland (seit 2008)**
    * **Inhalt:** Bevölkerungsstand pro Bezirk, detailliert aufgeschlüsselt nach Alter (5-Jahresgruppen), Geschlecht und Herkunft (9 Bundesländer + Ausland).
    * **Quelle:** Stadt Wien - OGD.

* **Datensatz B: Geburten pro Bezirk (seit 2002)**
    * **Inhalt:** Anzahl der Lebendgeburten pro Bezirk, Jahr und Geschlecht des Kindes.
    * **Quelle:** Stadt Wien - OGD - Bevölkerung zu Beginn & Geburten.

### 2. Implementierte Transformationsschritte (Preprocessing)
Um die Daten analysierbar zu machen, werden folgende Bereinigungen durchgeführt:

1.  **Extraction:** Direkter Abruf der aktuellsten `.csv`-Dateien über die Schnittstelle der Stadt Wien.
2.  **Standardisierung:**
    * Umcodierung der Bezirks-Kennziffern von `90100` (OGD-Standard) auf `1010` (Allgemeiner Standard).
    * Umbenennung kryptischer Spalten (z.B. `BGD` → `Burgenland`, `REF_YEAR` → `Jahr`).
3.  **Data Quality & Imputation:**
    * Auffüllen fehlender Werte (`NaN`) mit `0`.
    * Typ-Konvertierung in Integer (Ganzzahlen).
    * Sanity Check: Bereinigung unlogischer negativer Werte.
4.  **Loading:** Speicherung als `cleaned_population.csv` und `cleaned_births.csv` für die weitere Modellierung.


In [11]:
URL_BEVOELKERUNG = "https://www.wien.gv.at/gogv/l9ogdviebezpopsexage5stkcobgeoat102008f"
URL_GEBURTEN = "https://www.wien.gv.at/gogv/l9ogdviebezpopsexbir2002f"


def clean_bezirk(code):
    #Wandelt 90100 in 1010 um.
    try:
        code_str = str(code)
        # Prüfen ob Format 9xxxx
        if len(code_str) == 5 and code_str.startswith('9'):
            bez_num = code_str[1:3]
            return int(f"1{bez_num}0")
        return code 
    except:
        return 0

def run_data_pipeline():
    print("Pipeline gestartet: ")
    
    #1. EXTRACTION
    try:
        print("Lade Daten von data.wien.gv.at...")
        
        df_pop = pd.read_csv(URL_BEVOELKERUNG, sep=';', header=1, encoding='utf-8') #header 1 da erste Zeile Column Namen sind
        df_birth = pd.read_csv(URL_GEBURTEN, sep=';', header=1, encoding='utf-8')
    except Exception as e:
        print(f"Fehler beim Download: {e}")
        return None

    #2. TRANSFORMATION
    print(" Bereinige Bevölkerungsdaten...")
    
    df_pop = df_pop.rename(columns={
        'REF_YEAR': 'Jahr', 
        'DISTRICT_CODE': 'Bezirk_Roh', 
        'SEX': 'Geschlecht', 
        'AGE5': 'Alter_Gruppe',
        'BGD': 'Burgenland', 'KTN': 'Kaernten', 'NOE': 'Niederoesterreich',
        'OOE': 'Oberoesterreich', 'SBG': 'Salzburg', 'STK': 'Steiermark',
        'TIR': 'Tirol', 'VBG': 'Vorarlberg', 'VIE': 'Wien',
        'FOR': 'Ausland', 'UNK': 'Unbekannt'
    })

    df_pop['Bezirk'] = df_pop['Bezirk_Roh'].apply(clean_bezirk)

    #Fehlende Werte (NaN) durch 0 ersetzen und Datentypen (int) fixen
    cols_check = ['Burgenland', 'Kaernten', 'Niederoesterreich', 'Oberoesterreich', 
                  'Salzburg', 'Steiermark', 'Tirol', 'Vorarlberg', 'Wien', 
                  'Ausland', 'Unbekannt']
    
    df_pop[cols_check] = df_pop[cols_check].fillna(0).astype(int)
    
    #Negative Werte entfernen
    for col in cols_check:
        df_pop.loc[df_pop[col] < 0, col] = 0
        
    # Nur die sauberen Spalten behalten
    final_pop_cols = ['Jahr', 'Bezirk', 'Geschlecht', 'Alter_Gruppe'] + cols_check
    df_pop = df_pop[final_pop_cols]

    print("Bereinige Geburtsdaten...")
    # Geburten bereinigen
    df_birth = df_birth.rename(columns={
        'REF_YEAR': 'Jahr',
        'DISTRICT_CODE': 'Bezirk_Roh', 
        'SEX': 'Geschlecht',
        'BIR': 'Anzahl_Geburten'
    })
    
    df_birth['Bezirk'] = df_birth['Bezirk_Roh'].apply(clean_bezirk)
    df_birth['Anzahl_Geburten'] = df_birth['Anzahl_Geburten'].fillna(0).astype(int)
    
    
    df_birth = df_birth[['Jahr', 'Bezirk', 'Geschlecht', 'Anzahl_Geburten']]

    print("Speichere CSV Dateien...")
    
    df_pop.to_csv("cleaned_population.csv", index=False)
    df_birth.to_csv("cleaned_births.csv", index=False)
    
    print("Dateien 'cleaned_population.csv' und 'cleaned_births.csv' erstellt.")
    return df_pop, df_birth


df_pop_clean, df_birth_clean = run_data_pipeline()

print("\n--- Vorschau Bevölkerung ---")
display(df_pop_clean.head())

print("\n--- Vorschau Geburten ---")
display(df_birth_clean.head())

Pipeline gestartet: 
Lade Daten von data.wien.gv.at...
 Bereinige Bevölkerungsdaten...
Bereinige Geburtsdaten...
Speichere CSV Dateien...
Dateien 'cleaned_population.csv' und 'cleaned_births.csv' erstellt.

--- Vorschau Bevölkerung ---


,Jahr,Bezirk,Geschlecht,Alter_Gruppe,Burgenland,Kaernten,Niederoesterreich,Oberoesterreich,Salzburg,Steiermark,Tirol,Vorarlberg,Wien,Ausland,Unbekannt
0,2008,1010,1,1,0,1,9,1,0,4,0,0,207,39,0
1,2008,1010,1,2,0,1,3,2,1,1,0,0,202,44,0
2,2008,1010,1,3,0,2,7,4,1,0,1,2,198,40,1
3,2008,1010,1,4,2,2,7,3,2,3,1,2,201,72,3
4,2008,1010,1,5,2,1,18,6,3,6,4,1,253,138,2



--- Vorschau Geburten ---


,Jahr,Bezirk,Geschlecht,Anzahl_Geburten
0,2002,1010,1,58
1,2002,1010,2,72
2,2002,1020,1,490
3,2002,1020,2,491
4,2002,1030,1,370
